In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import os

def charger_dataset():
    """
    Charge le dataset Titanic
    """
    chemin = 'WEEK_2/DAY_3/DATASET/train.csv'
    return pd.read_csv(chemin)

In [10]:
def detecter_outliers_iqr(df, colonne):
    """
    Détecte les outliers en utilisant la méthode IQR
    """
    Q1 = df[colonne].quantile(0.25)
    Q3 = df[colonne].quantile(0.75)
    IQR = Q3 - Q1
    borne_inf = Q1 - 1.5 * IQR
    borne_sup = Q3 + 1.5 * IQR
    
    outliers = df[(df[colonne] < borne_inf) | (df[colonne] > borne_sup)][colonne]
    
    return outliers, borne_inf, borne_sup

In [12]:
def detecter_outliers_zscore(serie, seuil=3):
    """
    Détecte les outliers en utilisant le Z-score
    """
    serie = serie.dropna()  # Supprime les valeurs NaN
    z_scores = np.abs(stats.zscore(serie))
    return serie[z_scores > seuil]

In [15]:
def analyser_outliers():
    # Chargement des données
    print("1. Chargement du dataset...")
    df = charger_dataset()
    
    resultats = {}
    
    # Analyse des colonnes Age et Fare
    for colonne in ['Age', 'Fare']:
        print(f"\nAnalyse des outliers pour {colonne}:")
        
        # Méthode IQR
        outliers_iqr, borne_inf, borne_sup = detecter_outliers_iqr(df, colonne)
        print(f"\nMéthode IQR:")
        print(f"Nombre d'outliers: {len(outliers_iqr)}")
        print(f"Bornes: [{borne_inf:.2f}, {borne_sup:.2f}]")
        
        # Méthode Z-score
        outliers_zscore = detecter_outliers_zscore(df[colonne])
        print(f"\nMéthode Z-score (seuil=3):")
        print(f"Nombre d'outliers: {len(outliers_zscore)}")
        
        resultats[colonne] = {
            'iqr': (outliers_iqr, borne_inf, borne_sup),
            'zscore': outliers_zscore
        }
    
    return df, resultats

In [16]:
def traiter_outliers(df, resultats):
    """
    Traite les outliers avec capping et transformation log
    """
    df_traite = df.copy()
    
    for colonne in ['Age', 'Fare']:
        # Récupération des bornes IQR
        _, borne_inf, borne_sup = resultats[colonne]['iqr']
        
        # Capping
        df_traite[f'{colonne}_capped'] = df_traite[colonne].clip(borne_inf, borne_sup)
        
        # Log transformation pour Fare
        if colonne == 'Fare':
            df_traite[f'{colonne}_log'] = np.log1p(df_traite[colonne])
    
    return df_traite

In [7]:
def evaluer_impact(df_original, df_traite):
    """
    Évalue l'impact du traitement des outliers
    """
    for colonne in ['Age', 'Fare']:
        print(f"\nImpact sur {colonne}:")
        
        # Statistiques avant/après
        print("\nAvant traitement:")
        print(df_original[colonne].describe())
        print("\nAprès traitement:")
        print(df_traite[f'{colonne}_capped'].describe())
        
        # Nombre de valeurs modifiées
        n_modif = (df_original[colonne] != df_traite[f'{colonne}_capped']).sum()
        pct_modif = (n_modif / len(df_original)) * 100
        print(f"\nPourcentage de valeurs modifiées: {pct_modif:.2f}%")

In [17]:
def main():
    # Détection et analyse
    df_original, resultats = analyser_outliers()
    
    # Traitement
    df_traite = traiter_outliers(df_original, resultats)
    
    # Évaluation
    evaluer_impact(df_original, df_traite)
    
    print("\nTraitement des outliers terminé avec succès!")

if __name__ == "__main__":
    main()

1. Chargement du dataset...

Analyse des outliers pour Age:

Méthode IQR:
Nombre d'outliers: 11
Bornes: [-6.69, 64.81]

Méthode Z-score (seuil=3):
Nombre d'outliers: 2

Analyse des outliers pour Fare:

Méthode IQR:
Nombre d'outliers: 116
Bornes: [-26.72, 65.63]

Méthode Z-score (seuil=3):
Nombre d'outliers: 20

4. Évaluation de l'impact du traitement...

Impact sur Age:

Changement dans les statistiques:
count     0.000000
mean     -0.076418
std      -0.209832
min       0.000000
25%       0.000000
50%       0.000000
75%       0.000000
max     -15.187500
dtype: float64

Pourcentage de valeurs modifiées: 21.10%

Impact sur Fare:

Changement dans les statistiques:
count      0.000000
mean      -8.157395
std      -29.211804
min        0.000000
25%        0.000000
50%        0.000000
75%        0.000000
max     -446.694800
dtype: float64

Pourcentage de valeurs modifiées: 13.02%

Traitement des outliers terminé avec succès!
